In [1]:
import json
import numpy as np
import random
#from tqdm.auto import tqdm
from tqdm import tqdm
import itertools
import os
from copy import deepcopy
import matplotlib.pyplot as plt
import copy

In [2]:
# create a dictionary with entities(keys) and a assigning them an index number as dic value
def build_dicts(entities):
    entity2ind = dict()
    ind2entity = []
    for i in range(len(entities)):
        entity = entities[i]
        if not (entity in ind2entity):
            ind2entity.append(entity)
            entity2ind[entity] = len(ind2entity) - 1
    return ind2entity, entity2ind

# chose ration of dataset used for ID and OOD
def choose(arr, ratio_or_count):
    if type(ratio_or_count) == float:
        num = round(ratio_or_count*len(arr))
    elif type(ratio_or_count) == int:
        num = ratio_or_count
    else:
         assert False
    if num >= len(arr):
        return arr    # if we chose to take more data then in the array just take all the array
    rand_inds = np.random.choice(len(arr), num, replace=False).tolist()
    return [arr[i] for i in rand_inds]

# Splits an array into two parts test-train  
def split(arr, ratio_or_count):
    if type(ratio_or_count) == float:
        num = round(ratio_or_count*len(arr))
    elif type(ratio_or_count) == int:
        num = ratio_or_count
    else:
         assert False
    train, test = [], []
    rand_inds = np.random.choice(len(arr), num, replace=False).tolist()
    for i in tqdm(range(len(arr))):
        if i in rand_inds:
            train.append(arr[i])
        else:
            test.append(arr[i])
    return [train, test]

def form_items(c, t):
    #The join method concatenates all elements in the list c into a single string. For example, if c = ["<e_1>", "<r_1>"], then input_text will be "<e_1><r_1>".
    input_text = "".join(c)
    target_text = input_text + "".join([t, "</a>"]) # in  alist before for making more readable
    item = {
        "input_text": input_text,
        "target_text": target_text
    }
    return item

def form_items_SPLIT(c, t):
    #The join method concatenates all elements in the list c into a single string. For example, if c = ["<e_1>", "<r_1>"], then input_text will be "<e_1><r_1>".
    input_text = "".join(c+ ["<mask>"])
    target_text = "".join(c+ [t, "</a>"]) # in  alist before for making more readable
    item = {
        "input_text": input_text,
        "target_text": target_text
    }
    return item



def form_items_SPLIT_inferred_2(c, d, t, hop_1):
    # c = [ent, r1], d = r2, t = tail , hop_1 = b
    # Concatenate elements from the list `c` with `<mask>`, `d`, and `<mask>` correctly
    input_text = "".join(c + ["<mask>", d, "<mask>"])  # Concatenate c with <mask>, d, <mask>
    
    # Concatenate elements for the target_text
    target_text = "".join(c + [hop_1, "</a>", d, t, "</a>"])  # Concatenate c with hop_1, "</a>", d, t, "</a>"
    
    item = {
        "input_text": input_text,
        "target_text": target_text
    }
    
    return item

def form_items_SPLIT_inferred_3(c,hop_1 ,r_2,hop_2, r_3,  t  ):
    # Concatenate elements from the list `c` with `<mask>`, `d`, and `<mask>` correctly
    # c = [ent, r1], d = r2, e = r3, t = tail , hop_1 = b, hop_2 = c
    input_text = "".join(c + ["<mask>", r_2, "<mask>", r_3, "<mask>"])  # Concatenate c with <mask>, d, <mask>, e, <mask>
    
    # Concatenate elements for the target_text
    target_text = "".join(c + [hop_1, "</a>", r_2, hop_2, "</a>", r_3, t, "</a>"])  # Concatenate c with hop_1, "</a>", d, hop_2, "</a>", e, t, "</a>"
    
    item = {
        "input_text": input_text,
        "target_text": target_text
    }
    
    return item
   




#test_inferred_ood.append(form_items([ent, r1],hop_1= b , r2, t)) 

#t3_HOP_test_inferred_ood.append(form_items([ent, r1],hop_1=b, r2, hop_2=c,  r3, t_3))

In [3]:
form_items_SPLIT_inferred_3(["<e_189>","<r_7>"] ,hop_1= "<e_1>", r_2= "<r_2>",hop_2= "<e_2>", r_3= "<r_3>",  t= "<e_3>")

{'input_text': '<e_189><r_7><mask><r_2><mask><r_3><mask>',
 'target_text': '<e_189><r_7><e_1></a><r_2><e_2></a><r_3><e_3></a>'}

In [4]:
form_items_SPLIT_inferred_2(["<e_176>","<r_7>"] , "<r__1>", "<e_22>",hop_1= "<e_1>")

{'input_text': '<e_176><r_7><mask><r__1><mask>',
 'target_text': '<e_176><r_7><e_1></a><r__1><e_22></a>'}

In [5]:
def build_dataset(num_entities, num_relations, out_degree=20, split_train_inferred=False):
 
    # create a list with all entities names
    entities = ["<e_{}>".format(i) for i in range(num_entities)]
    # create a dictionary with entities as keys and index as values
    ind2entity, entity2ind = build_dicts(entities)

    # create a list with all relations names
    relations = ["<r_{}>".format(i) for i in range(num_relations)]
    # create a dictionary with relations as keys and index as values
    ind2relation, relation2ind = build_dicts(relations)

    #create atomic facts = dictionary head entity -> list of (relation, tail entity) pairs
    atomic_dict = dict()   # maps a head entity to a list of (r, t) pairs
    atomic_facts = []
    atomics = []

    for i in tqdm(range(num_entities)): #  it creates a progress bar that updates as the loop progresses
        # for each subject entity, randomly select some outgoing relations to some random object entity
        num_rows = out_degree
        # randomly select some relations (for each head entity), size=num_rows is the number of relations to be selected=20= out_degree for node
        selected_rows = np.random.choice(num_relations, size=num_rows, replace=False).tolist()
        for row_idx in selected_rows:
            col_idx = np.random.randint(num_entities)  # pick some random tail entity for each selected (h,r)
            h,r,t = ind2entity[i], ind2relation[row_idx], ind2entity[col_idx]
            atomic_facts.append(form_items([h, r], t))
            atomics.append((h,r,t))
            if h not in atomic_dict:  # add the head entity to the dictionary if it's not already there
                atomic_dict[h] = []
            # add the (r, t) pair to the list of pairs for this head entity. In this way a key (head)is associated to all the relations and tail entities
            atomic_dict[h].append((r, t))   
            ############################################################################################################################################  HERE end the ATOMIC FACTS CREATION!!!  ###############

    if not split_train_inferred:   # if we don't want to split the training set into ID and OOD as onlly ID is needed skip this part    NOOOOOOOOOOOO USE
        inferred_facts = []
        for ent in tqdm(entities):
            for (r1, b) in atomic_dict[ent]:  # for each (r1, b) pair associated with the head entity
                for (r2, t) in atomic_dict[b]:   # for each (r2, t) pair associated with the bridge entity
                    inferred_facts.append(form_items([ent, r1, r2], t))  # add the inferred fact to the list of inferred facts
        return entities, relations, atomic_facts, inferred_facts
    ################################################################################################################ continue with the split of the training set in ID and OOD
    # split ID/OOD
    OOD_ratio = 0.05          #atomic=list of tuples = all the atomic facts; atomics.append((h,r,t))
    OOD_facts, ID_facts = split(atomics, round(len(atomics)*OOD_ratio)) #split the atomic facts in ID and OOD
    OOD_facts, ID_facts = set(OOD_facts), set(ID_facts)                 #convert the lists OOD and ID in sets



    # create a list of atomic facts for ID and OOD "iteams" see form_items function
    # FROM A LIST OF TUPLES TO A LIST OF DICTIONARIES!!

#OOD_facts = [("<e_1>", "<r_1>", "<e_2>"), ("<e_3>", "<r_2>", "<e_4>")]
# transform in:
#ood_atomic_facts = [
#   {
#        "input_text": "<e_1><r_1>",
#        "target_text": "<e_1><r_1><e_2></a>"  --> in other words:"(h,r,t)" + "</a>"
#    },{"input ".. } ..]
  #################################################################################### create/convert the atomic in iteams way
    id_atomic_facts = [form_items([h, r], t) for (h,r,t) in ID_facts]
    ood_atomic_facts = [form_items([h, r], t) for (h,r,t) in OOD_facts]

    ### MIO SPLIT the hops 
    SPLIT_id_atomic_facts =   [form_items_SPLIT([h, r], t) for (h,r,t) in ID_facts]  # just add rhe mask to the  
    SPLIT_ood_atomic_facts =  [form_items_SPLIT([h, r], t) for (h,r,t) in OOD_facts]


    cc=0
    ########  HERE WE CREATE THE TRAIN AND TEST SETS FOR THE INFERENCE TASK ########
    #lets see what we train on and what we test on!

    train_inferred_facts, test_inferred_iid, test_inferred_ood = [], [], []
    t3_HOP_train_inferred_facts ,t3_HOP_test_inferred_iid , t3_HOP_test_inferred_ood = [], [], []      # 3-hop inference  (MY)

    # SPLIT the hops  adding <mask> to the input_text
    SPLIT_train_inferred_facts, SPLIT_test_inferred_iid, SPLIT_test_inferred_ood = [], [], []

    SPLIT_t3_HOP_train_inferred_facts ,SPLIT_t3_HOP_test_inferred_iid , SPLIT_t3_HOP_test_inferred_ood = [], [], []      # 3-hop inference  split

    #### mask the only change is that in the training we put the ood in the second hop but coming from mask,mask,r_2
    M_train_inferred_fact, M_test_inferred_iid, M_train_inferred_ood = [], [], []

    M_M_train_inferred_fact, M_test_inferred_iid, M_M_train_inferred_ood = [], [], []
    ID_M_M_train_inferred_fact, M_test_inferred_iid, ID_M_M_train_inferred_ood = [], [], []   # with also some ID (mask, Id)
    OOD_allM_M_train_inferred_fact =[] # is used for putting in the training all the OOD in the second hop (by passing the phi % that
    only_mask_ID=[]
    # is a great disadvantage for OOD is there are 200 OOD atomic vs 3800 ID in case of 200_20 Small. That means that the % to not get 
    #one OOD in the second hop is much greater)


    for ent in tqdm(entities):
        #for each entity we loop over all the stored out_edges (relations and tail entities)   basically each entity is stored like= h : [(r1, b), (r2, c), (r3, d)] 
        for (r1, b) in atomic_dict[ent]:
            for (r2, t) in atomic_dict[b]:   #now thre same looping but starting from the tail entity of before
                if (ent, r1, b) in OOD_facts or (b, r2, t) in OOD_facts:  # at least one of the two entities is in the OOD set

                    
                    #if the head entity and the tail entity are in the OOD set, then we add the fact to the test set(OOD)
                    if (ent, r1, b) in OOD_facts and (b, r2, t) in OOD_facts:
                        test_inferred_ood.append(form_items([ent, r1, r2], t)) #if both entities are in the OOD set, we add the fact to the OOD list

                        SPLIT_test_inferred_ood.append(form_items_SPLIT_inferred_2([ent, r1] , r2, t,hop_1= b))
                        
                        M_train_inferred_ood.append(form_items(["<mask>", "<mask>", r2], t))  # 
                        M_M_train_inferred_ood.append(form_items(["<mask>", b, r2], t))
                    

                        #form_items_SPLIT_inferred_2(["<e_176>","<r_7>"] , "<r__1>", "<e_22>",hop_1= "<e_1>")
                    #continue # it exits the current iteration and goes to the next one (if both are in OOD we haveadded to OOD list otherwise we don't care either) 
                
                else: 
                    #NOW if none of the two entities are in the OOD set we add the fact to the train set (randomly with a 99.95% probability)
                    if np.random.uniform() > 0.005:
             
                        train_inferred_facts.append(form_items([ent, r1, r2], t))
                        ID_M_M_train_inferred_fact.append(form_items(["<mask>", b, r2], t))  # ad some id with mask, id as OOD (last test)
                        only_mask_ID.append(form_items(["<mask>", b, r2], t))
                        SPLIT_train_inferred_facts.append(form_items_SPLIT_inferred_2([ent, r1] , r2, t,hop_1= b))
                    else:
                        test_inferred_iid.append(form_items([ent, r1, r2], t))
                        SPLIT_test_inferred_iid.append(form_items_SPLIT_inferred_2([ent, r1] , r2, t,hop_1= b))




                ###### 3-hop implementation
                for (r3, t_3) in atomic_dict[t]:  # for each (r3, t) pair associated with the tail entity
                    if (ent, r1, b) in OOD_facts or (b, r2, t) in OOD_facts or (t, r3, t_3) in OOD_facts:     #
                        #print(" or check 1\n")
                        if (ent, r1, b) in OOD_facts and (b, r2, t) in OOD_facts and (t, r3, t_3) in OOD_facts:   # check if all the entities are in the OOD set
                            #print(" or check 2\n")
                            t3_HOP_test_inferred_ood.append(form_items([ent, r1, r2, r3], t_3))
                            SPLIT_t3_HOP_test_inferred_ood.append(form_items_SPLIT_inferred_3([ent, r1] ,hop_1= b, r_2= r2,hop_2= t, r_3= r3,  t= t_3))
                        continue     
                        #if at least one of the entities is in the OOD set,(but not all) we skip the current iteration = discard the inferred fact 

                    #if none of the entities are in the OOD set, we add the fact to the train set (randomly with a 0.5 probability)
                    if np.random.uniform() > 0.005:
                        t3_HOP_train_inferred_facts.append(form_items([ent, r1, r2, r3], t_3))
                        SPLIT_t3_HOP_train_inferred_facts.append(form_items_SPLIT_inferred_3([ent, r1] ,hop_1= b, r_2= r2,hop_2= t, r_3= r3,  t= t_3))
                        
                    else:
                        t3_HOP_test_inferred_iid.append(form_items([ent, r1, r2, r3], t_3))
                        SPLIT_t3_HOP_test_inferred_iid.append(form_items_SPLIT_inferred_3([ent, r1] ,hop_1= b, r_2= r2,hop_2= t, r_3= r3,  t= t_3))
                               
            

    ####
    # Iterate through M_test_inferred_ood  (mask,mask)
    M_train_inferred_fact.extend(copy.deepcopy(train_inferred_facts))
    #M_train_inferred_fact.append(train_inferred_facts) #
    # we add to the training the mas,mask inferred ood
    for entry in M_train_inferred_ood:
        #if np.random.uniform() > 0.005:
        M_train_inferred_fact.append(entry)
    # (mask b)
    M_M_train_inferred_fact.extend(copy.deepcopy(train_inferred_facts))
    # we add to the training the mas,mask inferred ood ( so normal train + mask ood)
    for entry in M_M_train_inferred_ood:
        #if np.random.uniform() > 0.005:
        M_M_train_inferred_fact.append(entry)
    #(mak, B)  with also ID
    ID_M_M_train_inferred_fact.extend(copy.deepcopy(train_inferred_facts))
    # we add to the training the mas,mask inferred ood and id
    for entry in M_M_train_inferred_ood:
        #if np.random.uniform() > 0.005:
        ID_M_M_train_inferred_fact.append(entry)

    print("len mask B", len(M_M_train_inferred_fact))
    print("len IDD  mask B", len(ID_M_M_train_inferred_fact))
    print("only ood",len(M_M_train_inferred_ood))
    print("train inf",len( train_inferred_facts))


    return (entities, relations, id_atomic_facts, ood_atomic_facts, train_inferred_facts, test_inferred_iid, test_inferred_ood , t3_HOP_train_inferred_facts ,t3_HOP_test_inferred_iid , t3_HOP_test_inferred_ood ,
            
             SPLIT_id_atomic_facts , SPLIT_ood_atomic_facts, SPLIT_train_inferred_facts, SPLIT_test_inferred_iid, SPLIT_test_inferred_ood , SPLIT_t3_HOP_train_inferred_facts ,SPLIT_t3_HOP_test_inferred_iid , SPLIT_t3_HOP_test_inferred_ood , 
             M_train_inferred_fact, M_M_train_inferred_fact , ID_M_M_train_inferred_fact  , M_M_train_inferred_ood , only_mask_ID)
    
    

In [6]:
NUM_ENTITY_IN = 2000    # complete 2000
NUM_RELATION =  200     #complete 200   # 20
 
(train_entities, train_relations,
id_atomic_facts,         ood_atomic_facts,          train_inferred_facts,   test_inferred_iid, test_inferred_facts ,      t_3_train_inferred_facts, t_3_test_inferred_iid, t_3_test_inferred_facts,
SPLIT_id_atomic_facts , SPLIT_ood_atomic_facts, SPLIT_train_inferred_facts, SPLIT_test_inferred_iid, SPLIT_test_inferred_facts , SPLIT_t3_HOP_train_inferred_facts ,SPLIT_t3_HOP_test_inferred_iid , SPLIT_t3_HOP_test_inferred_facts,
M_train_inferred_fact, M_M_train_inferred_fact ,ID_M_M_train_inferred_fact , OOD_mask_B_all ,only_mask_ID)  = build_dataset(NUM_ENTITY_IN, NUM_RELATION, split_train_inferred=True)

#train_entities, train_relations are the same in all the cases


100%|██████████| 2000/2000 [01:05<00:00, 30.61it/s]


len mask B 720397
len IDD  mask B 1438727
only ood 2067
train inf 718330


In [7]:
test_unione=train_inferred_facts + OOD_mask_B_all
len(test_unione)
for phi in [12.6]:  #phi is the ratio (Inferred facts / Atomic facts)
    train_inferred_facts_ds_test_2 = choose(test_unione, round(phi * len(id_atomic_facts))) #downsampling the train_inferred_facts (ID)
    print("aa",len(train_inferred_facts_ds_test_2))
data_2=train_inferred_facts_ds_test_2
mask_count_2= sum(item["input_text"].count("<mask>") for item in data_2)

print(f"The tag <mask> appears {mask_count_2} times in the input text.")



aa 478800
The tag <mask> appears 1361 times in the input text.


In [8]:
for phi in [12.6]:  #phi is the ratio (Inferred facts / Atomic facts)

    train_inferred_facts_ds_test_1 = choose(M_M_train_inferred_fact, round(phi * len(id_atomic_facts))) #downsampling the train_inferred_facts (ID)
    print(len(train_inferred_facts_ds_test_1))
data_1=train_inferred_facts_ds_test_1
mask_count_1= sum(item["input_text"].count("<mask>") for item in data_1)

print(f"The tag <mask> appears {mask_count_1} times in the input text.")



478800
The tag <mask> appears 1400 times in the input text.


In [9]:
for phi in [12.6]:  #phi is the ratio (Inferred facts / Atomic facts)

    train_inferred_facts_ds_test = choose(ID_M_M_train_inferred_fact, round(phi * len(id_atomic_facts))) #downsampling the train_inferred_facts (ID)
    print(len(train_inferred_facts_ds_test))

data=train_inferred_facts_ds_test
mask_count = sum(item["input_text"].count("<mask>") for item in data)

print(f"The tag <mask> appears {mask_count} times in the input text.")



478800
The tag <mask> appears 239586 times in the input text.


In [10]:
# Check if any input text contains <mask> and print it
for fact in train_inferred_facts:
    if '<mask>' in fact['input_text']:
        print(fact['input_text'])

 Explanation of build dataset output and renaming for dataset creation:
 > train_entities = list of all entities [e_0, e_1, e_2, ...]

>train_relation=  list of all relations [r_0, r_1, r_2, ...]

>id_atomic_facts/ood_atomic_facts=   list of atomic facts =list of dictionaries with 2keys

    input_text(h,r) and target_text(h,r,t)  for each fact 
    
>train_inferred_facts (= training inferred only on ID) =  ...

>test_inferred_iid/test_inferred_facts(=test_inferred_ood)  =  ...

In [11]:
# vocab is a list of all the entities and relations in the dataset =list of strings
vocab = []
vocab = vocab + train_entities + train_relations
# special tokens added to the vocabulary for the model to understand the input and output format
vocab = vocab + ["<mask>", "<sep>", "<a>", "</a>", "<q>", "</q>"]
assert len(vocab) == len(set(vocab))
print("vocab size:", len(vocab))

vocab size: 2206


In [12]:
test_size = 3000       #3000 complete
id_atomic_facts_ds = choose(id_atomic_facts, test_size)
ood_atomic_facts_ds = choose(ood_atomic_facts, test_size)
test_inferred_iid_ds = choose(test_inferred_iid, test_size) #IID
test_inferred_facts_ds = choose(test_inferred_facts, test_size) #OOD

all_atomics = id_atomic_facts + ood_atomic_facts
len(all_atomics)

40000

In [13]:
# downsampling train_inferred
for phi in [18.0,12.6,9.0,7.2,5.4,3.6]:  #phi is the ratio (Inferred facts / Atomic facts)
    dataset_name = "composition.{}.{}.{}".format(NUM_ENTITY_IN, NUM_RELATION, phi)
    os.makedirs("data_MIO_FINAL/{}".format(dataset_name), exist_ok=True)
    train_inferred_facts_ds = choose(train_inferred_facts, round(phi * len(id_atomic_facts))) #downsampling the train_inferred_facts (ID)

    probes = []
    for item in id_atomic_facts_ds:
        probes.append(deepcopy(item))
        probes[-1]["type"] = "id_atomic"
    
    for item in ood_atomic_facts_ds:
        probes.append(deepcopy(item))
        probes[-1]["type"] = "ood_atomic"

    for item in choose(train_inferred_facts_ds, test_size):
        probes.append(deepcopy(item))
        probes[-1]['type'] = 'train_inferred'

    for item in test_inferred_iid:
        probes.append(deepcopy(item))
        probes[-1]['type'] = 'test_inferred_iid'

    for item in test_inferred_facts_ds:
        probes.append(deepcopy(item))
        probes[-1]["type"] = "test_inferred_ood"

    # save the dataset Train, Test, Valid
    #train.json = all_atomics + train_inferred_facts_ds
    with open("data_MIO_FINAL/{}/train.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(all_atomics + train_inferred_facts_ds, f)
    #valid.json (only test_inferred_ OOD  why??? for what is that good?) 
    with open("data_MIO_FINAL/{}/valid.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(test_inferred_facts_ds, f)

    # my validation with the test_inferred_iid    
    with open("data_MIO_FINAL/{}/valid_iid.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(test_inferred_iid_ds, f)

    #test.json = probes (all the atomic facts and the inferred facts how I know are not used in training??)
    with open("data_MIO_FINAL/{}/test.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(probes, f)
    # all vocab in the dataset (entities and relations) + special tokens
    with open("data_MIO_FINAL/{}/vocab.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(vocab, f)

Now to add the info from the mask

In [14]:
# mask B augemnted with some OOD
for phi in [18.0,12.6,9.0,7.2,5.4,3.6]:  #phi is the ratio (Inferred facts / Atomic facts)
    dataset_name = "mask_B_composition_OOD_some.{}.{}.{}".format(NUM_ENTITY_IN, NUM_RELATION, phi)
    os.makedirs("data_MIO_FINAL/{}".format(dataset_name), exist_ok=True)
    M_M_train_inferred_facts_ds = choose(M_M_train_inferred_fact, round(phi * len(id_atomic_facts))) #downsampling the train_inferred_facts (ID)
    # augment the mask OOD in train (make sure they are not discared from the phi. However still this OOD_b_all is not really ALL!!!)
    for ood_ in OOD_mask_B_all:
        M_M_train_inferred_facts_ds.append(ood_)
    probes = []
    for item in id_atomic_facts_ds:
        probes.append(deepcopy(item))
        probes[-1]["type"] = "id_atomic"
    
    for item in ood_atomic_facts_ds:
        probes.append(deepcopy(item))
        probes[-1]["type"] = "ood_atomic"

    for item in choose(M_M_train_inferred_facts_ds, test_size):
        probes.append(deepcopy(item))
        probes[-1]['type'] = 'train_inferred'

    for item in test_inferred_iid:
        probes.append(deepcopy(item))
        probes[-1]['type'] = 'test_inferred_iid'

    for item in test_inferred_facts_ds:
        probes.append(deepcopy(item))
        probes[-1]["type"] = "test_inferred_ood"

    # save the dataset Train, Test, Valid
    #train.json = all_atomics + train_inferred_facts_ds
    with open("data_MIO_FINAL/{}/train.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(all_atomics + M_M_train_inferred_facts_ds, f)
    #valid.json (only test_inferred_ OOD  why??? for what is that good?) 
    with open("data_MIO_FINAL/{}/valid.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(test_inferred_facts_ds, f)

    # my validation with the test_inferred_iid    
    with open("data_MIO_FINAL/{}/valid_iid.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(test_inferred_iid_ds, f)

    #test.json = probes (all the atomic facts and the inferred facts how I know are not used in training??)
    with open("data_MIO_FINAL/{}/test.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(probes, f)
    # all vocab in the dataset (entities and relations) + special tokens
    with open("data_MIO_FINAL/{}/vocab.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(vocab, f)

### her we make as before only OOD but we want to make the mask OOD =24 K  = half of the overall training inferred (otherwise only with aslo mask ID we have a right proportion between the normal iferred and the mask_B )

In [15]:
M_M_train_inferred_facts_ds = choose(M_M_train_inferred_fact, round(phi * len(id_atomic_facts)/2)) #downsampling the train_inferred_facts (ID)
print(len(M_M_train_inferred_facts_ds))

68400


In [16]:
# all OOD in 2-hop
OOD_2_HOP_all=[]

for fact in ood_atomic_facts:
    target_text = fact['target_text']
    parts = target_text.split('>')
    
    # Extract the first, second, and third parts
    b = parts[0] + '>'
    r2 = parts[1] + '>'
    t = parts[2] + '>'
    
    # Print the extracted parts
    #print(f"b: {b}, r: {r}, t: {t}")
    OOD_2_HOP_all.append(form_items(["<mask>", b, r2], t))
print(len(OOD_2_HOP_all))


# Now we extend the list to 23 k

# Calculate how many full repetitions are needed
repetitions = 23000 // len(OOD_2_HOP_all)

# Extend the list by repeating it and slicing to exactly 23,000 elements
OOD_2_HOP_all_23K_extended = (OOD_2_HOP_all * (repetitions + 1))[:23000]
print(len(OOD_2_HOP_all_23K_extended))

2000
23000


In [17]:
# mask B augemnted with ALL mask_B_OOD  until reach 23 K on a dataset of inferred+mask_B of  around 50K  ()
for phi in [18.0,12.6,9.0,7.2,5.4,3.6]:  #phi is the ratio (Inferred facts / Atomic facts)
    dataset_name = "23K_mask_B_composition_OOD_some.{}.{}.{}".format(NUM_ENTITY_IN, NUM_RELATION, phi)
    os.makedirs("data_MIO_FINAL/{}".format(dataset_name), exist_ok=True)
    M_M_train_inferred_facts_ds = choose(M_M_train_inferred_fact, round(phi * len(id_atomic_facts)/2)) #downsampling the train_inferred_facts  (we divede with 2  so we  have arounf 23k and we will add other 23 k from maks_b_ OOD
    # augment the mask OOD in train (make sure they are not discared from the phi. However still this OOD_b_all is not really ALL!!!)
    print("len FIRST of the total training inferred ",len( M_M_train_inferred_facts_ds))
    for ood_ in OOD_2_HOP_all_23K_extended:
        M_M_train_inferred_facts_ds.append(ood_)
    print("len of the total training inferred + mask_B_OOD",len( M_M_train_inferred_facts_ds),"\n")
    probes = []
    for item in id_atomic_facts_ds:
        probes.append(deepcopy(item))
        probes[-1]["type"] = "id_atomic"
    
    for item in ood_atomic_facts_ds:
        probes.append(deepcopy(item))
        probes[-1]["type"] = "ood_atomic"

    for item in choose(M_M_train_inferred_facts_ds, test_size):
        probes.append(deepcopy(item))
        probes[-1]['type'] = 'train_inferred'

    for item in test_inferred_iid:
        probes.append(deepcopy(item))
        probes[-1]['type'] = 'test_inferred_iid'

    for item in test_inferred_facts_ds:
        probes.append(deepcopy(item))
        probes[-1]["type"] = "test_inferred_ood"

    # save the dataset Train, Test, Valid
    #train.json = all_atomics + train_inferred_facts_ds
    with open("data_MIO_FINAL/{}/train.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(all_atomics + M_M_train_inferred_facts_ds, f)
    #valid.json (only test_inferred_ OOD  why??? for what is that good?) 
    with open("data_MIO_FINAL/{}/valid.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(test_inferred_facts_ds, f)

    # my validation with the test_inferred_iid    
    with open("data_MIO_FINAL/{}/valid_iid.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(test_inferred_iid_ds, f)

    #test.json = probes (all the atomic facts and the inferred facts how I know are not used in training??)
    with open("data_MIO_FINAL/{}/test.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(probes, f)
    # all vocab in the dataset (entities and relations) + special tokens
    with open("data_MIO_FINAL/{}/vocab.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(vocab, f)

len FIRST of the total training inferred  342000
len of the total training inferred + mask_B_OOD 365000 

len FIRST of the total training inferred  239400
len of the total training inferred + mask_B_OOD 262400 

len FIRST of the total training inferred  171000
len of the total training inferred + mask_B_OOD 194000 

len FIRST of the total training inferred  136800
len of the total training inferred + mask_B_OOD 159800 

len FIRST of the total training inferred  102600
len of the total training inferred + mask_B_OOD 125600 

len FIRST of the total training inferred  68400
len of the total training inferred + mask_B_OOD 91400 



In [18]:
# mask B normal not augmented
for phi in [18.0,12.6,9.0,7.2,5.4,3.6]:  #phi is the ratio (Inferred facts / Atomic facts)
    dataset_name = "mask_B_composition.{}.{}.{}".format(NUM_ENTITY_IN, NUM_RELATION, phi)
    os.makedirs("data_MIO_FINAL/{}".format(dataset_name), exist_ok=True)
    M_M_train_inferred_facts_ds_not_augmented = choose(M_M_train_inferred_fact, round(phi * len(id_atomic_facts))) #downsampling the train_inferred_facts (ID)

    probes = []
    for item in id_atomic_facts_ds:
        probes.append(deepcopy(item))
        probes[-1]["type"] = "id_atomic"
    
    for item in ood_atomic_facts_ds:
        probes.append(deepcopy(item))
        probes[-1]["type"] = "ood_atomic"

    for item in choose(M_M_train_inferred_facts_ds_not_augmented, test_size):
        probes.append(deepcopy(item))
        probes[-1]['type'] = 'train_inferred'

    for item in test_inferred_iid:
        probes.append(deepcopy(item))
        probes[-1]['type'] = 'test_inferred_iid'

    for item in test_inferred_facts_ds:
        probes.append(deepcopy(item))
        probes[-1]["type"] = "test_inferred_ood"

    # save the dataset Train, Test, Valid
    #train.json = all_atomics + train_inferred_facts_ds
    with open("data_MIO_FINAL/{}/train.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(all_atomics + M_M_train_inferred_facts_ds_not_augmented, f)
    #valid.json (only test_inferred_ OOD  why??? for what is that good?) 
    with open("data_MIO_FINAL/{}/valid.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(test_inferred_facts_ds, f)

    # my validation with the test_inferred_iid    
    with open("data_MIO_FINAL/{}/valid_iid.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(test_inferred_iid_ds, f)

    #test.json = probes (all the atomic facts and the inferred facts how I know are not used in training??)
    with open("data_MIO_FINAL/{}/test.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(probes, f)
    # all vocab in the dataset (entities and relations) + special tokens
    with open("data_MIO_FINAL/{}/vocab.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(vocab, f)

        

case mask, B with also some  ID (mask id) to help the model to connect the generalization information

In [19]:
# ID

# downsampling train_inferred
for phi in [18.0,12.6,9.0,7.2,5.4,3.6]:  #phi is the ratio (Inferred facts / Atomic facts)
    dataset_name = "ID_mask_B_composition.{}.{}.{}".format(NUM_ENTITY_IN, NUM_RELATION, phi)
    os.makedirs("data_MIO_FINAL/{}".format(dataset_name), exist_ok=True)
    ID_M_M_train_inferred_facts_ds = choose(ID_M_M_train_inferred_fact, round(phi * len(id_atomic_facts))) #downsampling the train_inferred_facts (ID)
    print("len ID normal", len(ID_M_M_train_inferred_facts_ds))
    probes = []
    for item in id_atomic_facts_ds:
        probes.append(deepcopy(item))
        probes[-1]["type"] = "id_atomic"
    
    for item in ood_atomic_facts_ds:
        probes.append(deepcopy(item))
        probes[-1]["type"] = "ood_atomic"

    for item in choose(ID_M_M_train_inferred_facts_ds, test_size):
        probes.append(deepcopy(item))
        probes[-1]['type'] = 'train_inferred'

    for item in test_inferred_iid:
        probes.append(deepcopy(item))
        probes[-1]['type'] = 'test_inferred_iid'

    for item in test_inferred_facts_ds:
        probes.append(deepcopy(item))
        probes[-1]["type"] = "test_inferred_ood"

    # save the dataset Train, Test, Valid
    #train.json = all_atomics + train_inferred_facts_ds
    with open("data_MIO_FINAL/{}/train.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(all_atomics + ID_M_M_train_inferred_facts_ds, f)
    #valid.json (only test_inferred_ OOD  why??? for what is that good?) 
    with open("data_MIO_FINAL/{}/valid.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(test_inferred_facts_ds, f)

    # my validation with the test_inferred_iid    
    with open("data_MIO_FINAL/{}/valid_iid.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(test_inferred_iid_ds, f)

    #test.json = probes (all the atomic facts and the inferred facts how I know are not used in training??)
    with open("data_MIO_FINAL/{}/test.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(probes, f)
    # all vocab in the dataset (entities and relations) + special tokens
    with open("data_MIO_FINAL/{}/vocab.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(vocab, f)

len ID normal 684000
len ID normal 478800
len ID normal 342000
len ID normal 273600
len ID normal 205200
len ID normal 136800


In [20]:
for phi in [18.0,12.6,9.0,7.2,5.4,3.6]:  #phi is the ratio (Inferred facts / Atomic facts)
    dataset_name = "ID_mask_B_composition.{}.{}.{}".format(NUM_ENTITY_IN, NUM_RELATION, phi)
    os.makedirs("data_MIO_FINAL/{}".format(dataset_name), exist_ok=True)
    ID_M_M_train_inferred_facts_ds = choose(ID_M_M_train_inferred_fact, round(phi * len(id_atomic_facts))) #downsampling the train_inferred_facts (ID)
len(ID_M_M_train_inferred_facts_ds)

136800

Now the case wehere all OOD (mask,B_ood) are placed in the training. This is done bc the ID atomic are 3800 and OOD atomic only 200. This lead that when we apply the phi ration the % that one OOD does not appear in the second hop (or appears only rarely) affects the generalization for OOD. Instead the IID appear in IID inferred with much more cases, that means that by downsamplig with phi, it affect each much less. TO notice that with (200 OOD each OOD appear at maximum 199 times as second hop, The ID 3799). -case for Small dataset 200wnriries 20 relations

In [21]:
# all OOD in 2-hop
OOD_2_HOP_all=[]

for fact in ood_atomic_facts:
    target_text = fact['target_text']
    parts = target_text.split('>')
    
    # Extract the first, second, and third parts
    b = parts[0] + '>'
    r2 = parts[1] + '>'
    t = parts[2] + '>'
    
    # Print the extracted parts
    #print(f"b: {b}, r: {r}, t: {t}")
    OOD_2_HOP_all.append(form_items(["<mask>", b, r2], t))

In [22]:




# ID con additiion of REALLY ALL OOD

# downsampling train_inferred
for phi in [18.0,12.6,9.0,7.2,5.4,3.6]:  #phi is the ratio (Inferred facts / Atomic facts)
    dataset_name = "ID_mask_B_composition_ALL_OOD.{}.{}.{}".format(NUM_ENTITY_IN, NUM_RELATION, phi)
    os.makedirs("data_MIO_FINAL/{}".format(dataset_name), exist_ok=True)
    ID_M_M_train_inferred_facts_ds_ALL_OOD = choose(ID_M_M_train_inferred_fact, round(phi * len(id_atomic_facts))) #downsampling the train_inferred_facts (ID)

    for ood_ in OOD_2_HOP_all:
        ID_M_M_train_inferred_facts_ds_ALL_OOD.append(ood_)

    print(" ID WITH ALL OOD addition:", len(ID_M_M_train_inferred_facts_ds_ALL_OOD))

    probes = []
    for item in id_atomic_facts_ds:
        probes.append(deepcopy(item))
        probes[-1]["type"] = "id_atomic"
    
    for item in ood_atomic_facts_ds:
        probes.append(deepcopy(item))
        probes[-1]["type"] = "ood_atomic"

    for item in choose(ID_M_M_train_inferred_facts_ds_ALL_OOD, test_size):
        probes.append(deepcopy(item))
        probes[-1]['type'] = 'train_inferred'

    for item in test_inferred_iid:
        probes.append(deepcopy(item))
        probes[-1]['type'] = 'test_inferred_iid'

    for item in test_inferred_facts_ds:
        probes.append(deepcopy(item))
        probes[-1]["type"] = "test_inferred_ood"

    # save the dataset Train, Test, Valid
    #train.json = all_atomics + train_inferred_facts_ds
    with open("data_MIO_FINAL/{}/train.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(all_atomics + ID_M_M_train_inferred_facts_ds_ALL_OOD, f)
    #valid.json (only test_inferred_ OOD  why??? for what is that good?) 
    with open("data_MIO_FINAL/{}/valid.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(test_inferred_facts_ds, f)

    # my validation with the test_inferred_iid    
    with open("data_MIO_FINAL/{}/valid_iid.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(test_inferred_iid_ds, f)

    #test.json = probes (all the atomic facts and the inferred facts how I know are not used in training??)
    with open("data_MIO_FINAL/{}/test.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(probes, f)
    # all vocab in the dataset (entities and relations) + special tokens
    with open("data_MIO_FINAL/{}/vocab.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(vocab, f)

 ID WITH ALL OOD addition: 686000
 ID WITH ALL OOD addition: 480800
 ID WITH ALL OOD addition: 344000
 ID WITH ALL OOD addition: 275600
 ID WITH ALL OOD addition: 207200
 ID WITH ALL OOD addition: 138800


In [23]:
ID_inferred_chosen=choose(only_mask_ID,200)
len(ID_inferred_chosen)
ID_inferred_chosen

[{'input_text': '<mask><e_1176><r_47>',
  'target_text': '<mask><e_1176><r_47><e_344></a>'},
 {'input_text': '<mask><e_735><r_79>',
  'target_text': '<mask><e_735><r_79><e_1448></a>'},
 {'input_text': '<mask><e_468><r_111>',
  'target_text': '<mask><e_468><r_111><e_1311></a>'},
 {'input_text': '<mask><e_245><r_160>',
  'target_text': '<mask><e_245><r_160><e_586></a>'},
 {'input_text': '<mask><e_1404><r_178>',
  'target_text': '<mask><e_1404><r_178><e_230></a>'},
 {'input_text': '<mask><e_1507><r_79>',
  'target_text': '<mask><e_1507><r_79><e_103></a>'},
 {'input_text': '<mask><e_1864><r_110>',
  'target_text': '<mask><e_1864><r_110><e_1871></a>'},
 {'input_text': '<mask><e_148><r_133>',
  'target_text': '<mask><e_148><r_133><e_451></a>'},
 {'input_text': '<mask><e_1576><r_165>',
  'target_text': '<mask><e_1576><r_165><e_259></a>'},
 {'input_text': '<mask><e_1276><r_77>',
  'target_text': '<mask><e_1276><r_77><e_1117></a>'},
 {'input_text': '<mask><e_341><r_153>',
  'target_text': '<mas

In [24]:
some_ID_M_M_train_inferred_facts_ds_ALL_OOD_test = choose(M_M_train_inferred_fact, round(8 * len(id_atomic_facts))) #downsampling the train_inferred_facts (ID)
len(some_ID_M_M_train_inferred_facts_ds_ALL_OOD_test)

some_ID_M_M_train_inferred_facts_ds_ALL_OOD_test.extend(ID_inferred_chosen)
len(some_ID_M_M_train_inferred_facts_ds_ALL_OOD_test)



304200

With ALL OOD doubel and some ID

In [25]:




# ID con additiion of REALLY ALL OOD

# downsampling train_inferred
for phi in [18.0,12.6,9.0,7.2,5.4,3.6]:  #phi is the ratio (Inferred facts / Atomic facts)
    dataset_name = "some_ID_mask_B_composition_ALL_OOD.{}.{}.{}".format(NUM_ENTITY_IN, NUM_RELATION, phi)
    os.makedirs("data_MIO_FINAL/{}".format(dataset_name), exist_ok=True)
    some_ID_M_M_train_inferred_facts_ds_ALL_OOD = choose(M_M_train_inferred_fact, round(phi * len(id_atomic_facts))) #downsampling the train_inferred_facts (ID)

    for ood_ in OOD_2_HOP_all:
        some_ID_M_M_train_inferred_facts_ds_ALL_OOD.append(ood_)
    ID_inferred_chosen=choose(only_mask_ID,400)
    some_ID_M_M_train_inferred_facts_ds_ALL_OOD.extend(ID_inferred_chosen)

    print(" some ID WITH ALL OOD addition:", len(some_ID_M_M_train_inferred_facts_ds_ALL_OOD))

    probes = []
    for item in id_atomic_facts_ds:
        probes.append(deepcopy(item))
        probes[-1]["type"] = "id_atomic"
    
    for item in ood_atomic_facts_ds:
        probes.append(deepcopy(item))
        probes[-1]["type"] = "ood_atomic"

    for item in choose(some_ID_M_M_train_inferred_facts_ds_ALL_OOD, test_size):
        probes.append(deepcopy(item))
        probes[-1]['type'] = 'train_inferred'

    for item in test_inferred_iid:
        probes.append(deepcopy(item))
        probes[-1]['type'] = 'test_inferred_iid'

    for item in test_inferred_facts_ds:
        probes.append(deepcopy(item))
        probes[-1]["type"] = "test_inferred_ood"

    # save the dataset Train, Test, Valid
    #train.json = all_atomics + train_inferred_facts_ds
    with open("data_MIO_FINAL/{}/train.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(all_atomics +some_ID_M_M_train_inferred_facts_ds_ALL_OOD, f)
    #valid.json (only test_inferred_ OOD  why??? for what is that good?) 
    with open("data_MIO_FINAL/{}/valid.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(test_inferred_facts_ds, f)

    # my validation with the test_inferred_iid    
    with open("data_MIO_FINAL/{}/valid_iid.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(test_inferred_iid_ds, f)

    #test.json = probes (all the atomic facts and the inferred facts how I know are not used in training??)
    with open("data_MIO_FINAL/{}/test.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(probes, f)
    # all vocab in the dataset (entities and relations) + special tokens
    with open("data_MIO_FINAL/{}/vocab.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(vocab, f)

 some ID WITH ALL OOD addition: 686400
 some ID WITH ALL OOD addition: 481200
 some ID WITH ALL OOD addition: 344400
 some ID WITH ALL OOD addition: 276000
 some ID WITH ALL OOD addition: 207600
 some ID WITH ALL OOD addition: 139200


In [26]:
print(" ID WITH ALL OOD addition:", len(ID_M_M_train_inferred_facts_ds))

 ID WITH ALL OOD addition: 136800


In [27]:
a

b
v

NameError: name 'a' is not defined

## Without double hops
only mask,B_id
mask,B_OOD

In [ ]:
OOD_mask_B_all ,only_mask_ID

In [ ]:




# ID con additiion of REALLY ALL OOD

# downsampling train_inferred
for phi in [18.0,12.6,9.0,7.2,5.4,3.6]:  #phi is the ratio (Inferred facts / Atomic facts)
    dataset_name = "no_2_hop_with_some.{}.{}.{}".format(NUM_ENTITY_IN, NUM_RELATION, phi)
    os.makedirs("data_MIO_FINAL/{}".format(dataset_name), exist_ok=True)
    no_2_hop_ds = choose(only_mask_ID, round(phi * len(id_atomic_facts))) #downsampling the train_inferred_facts (ID)

    for ood_ in OOD_2_HOP_all:
        no_2_hop_ds.append(ood_)
    for ood_ in OOD_2_HOP_all:
        no_2_hop_ds.append(ood_)
    ID_inferred_chosen_1=choose(train_inferred_facts,400)
    no_2_hop_ds.extend(ID_inferred_chosen_1)
   

    print(" NO 2-hop only mask_B:", len(no_2_hop_ds))

    probes = []
    for item in id_atomic_facts_ds:
        probes.append(deepcopy(item))
        probes[-1]["type"] = "id_atomic"
    
    for item in ood_atomic_facts_ds:
        probes.append(deepcopy(item))
        probes[-1]["type"] = "ood_atomic"

    for item in choose(no_2_hop_ds, test_size):
        probes.append(deepcopy(item))
        probes[-1]['type'] = 'train_inferred'

    for item in test_inferred_iid:
        probes.append(deepcopy(item))
        probes[-1]['type'] = 'test_inferred_iid'

    for item in test_inferred_facts_ds:
        probes.append(deepcopy(item))
        probes[-1]["type"] = "test_inferred_ood"

    # save the dataset Train, Test, Valid
    #train.json = all_atomics + train_inferred_facts_ds
    with open("data_MIO_FINAL/{}/train.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(all_atomics +no_2_hop_ds, f)
    #valid.json (only test_inferred_ OOD  why??? for what is that good?) 
    with open("data_MIO_FINAL/{}/valid.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(test_inferred_facts_ds, f)

    # my validation with the test_inferred_iid    
    with open("data_MIO_FINAL/{}/valid_iid.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(test_inferred_iid_ds, f)

    #test.json = probes (all the atomic facts and the inferred facts how I know are not used in training??)
    with open("data_MIO_FINAL/{}/test.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(probes, f)
    # all vocab in the dataset (entities and relations) + special tokens
    with open("data_MIO_FINAL/{}/vocab.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(vocab, f)

 NO 2-hop only mask_B: 69200
 NO 2-hop only mask_B: 48680
 NO 2-hop only mask_B: 35000
 NO 2-hop only mask_B: 28160
 NO 2-hop only mask_B: 21320
 NO 2-hop only mask_B: 14480


stop here for just 2-hop normal

### SPLIT HOPS 2

In [ ]:
# Atomic facts are slightly differently written (mask) for the SPLIT and NO split case but are the same for 2 ir 3 hop

test_size= 300
SPLIT_id_atomic_facts_ds = choose(SPLIT_id_atomic_facts, test_size)
SPLIT_ood_atomic_facts_ds = choose(SPLIT_ood_atomic_facts, test_size)
SPLIT_test_inferred_iid_ds = choose(SPLIT_test_inferred_iid, test_size) #IID
SPLIT_test_inferred_facts_ds = choose(SPLIT_test_inferred_facts, test_size) #OOD

SPLIT_all_atomics = SPLIT_id_atomic_facts + SPLIT_ood_atomic_facts
len(SPLIT_all_atomics)



In [ ]:
# downsampling train_inferred

for phi in [18.0,12.6,9.0,7.2,5.4,3.6]:  
    dataset_name = "SPLIT_composition.{}.{}.{}".format(NUM_ENTITY_IN, NUM_RELATION, phi)
    os.makedirs("data_MIO_mask_NEW_correct/{}".format(dataset_name), exist_ok=True)
    SPLIT_train_inferred_facts_ds = choose(SPLIT_train_inferred_facts, round(phi * len(SPLIT_id_atomic_facts))) #downsampling the train_inferred_facts (ID)

    SPLIT_probes = []
    for item in SPLIT_id_atomic_facts_ds:
        SPLIT_probes.append(deepcopy(item))
        SPLIT_probes[-1]["type"] = "id_atomic"

    for item in SPLIT_ood_atomic_facts_ds:
        SPLIT_probes.append(deepcopy(item))
        SPLIT_probes[-1]["type"] = "ood_atomic"

    for item in choose(SPLIT_train_inferred_facts_ds, test_size):
        SPLIT_probes.append(deepcopy(item))
        SPLIT_probes[-1]['type'] = 'train_inferred'

    for item in SPLIT_test_inferred_iid:
        SPLIT_probes.append(deepcopy(item))
        SPLIT_probes[-1]['type'] = 'test_inferred_iid'

    for item in SPLIT_test_inferred_facts_ds:
        SPLIT_probes.append(deepcopy(item))
        SPLIT_probes[-1]["type"] = "test_inferred_ood"

    # save the dataset Train, Test, Valid
    #train.json = all_atomics + train_inferred_facts_ds
    with open("data_MIO_mask_NEW_correct/{}/train.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(SPLIT_all_atomics + SPLIT_train_inferred_facts_ds, f)
    #valid.json (only test_inferred_ OOD  why??? for what is that good?)
    with open("data_MIO_mask_NEW_correct/{}/valid.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(SPLIT_test_inferred_facts_ds, f)
    
    # my validation with the test_inferred_iid
    with open("data_MIO_mask_NEW_correct/{}/valid_iid.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(SPLIT_test_inferred_iid_ds, f)

    #test.json = 
    with open("data_MIO_mask_NEW_correct/{}/test.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(SPLIT_probes, f)
    # all vocab in the dataset (entities and relations) + special tokens
    with open("data_MIO_mask_NEW_correct/{}/vocab.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(vocab, f)
    
    

# 3-hop inference save

the vocab is actually the same

In [ ]:

# vocab is a list of all the entities and relations in the dataset =list of strings
vocab = []
print(vocab)   # to test the vocab list is refreshed
vocab = vocab + train_entities + train_relations
# special tokens added to the vocabulary for the model to understand the input and output format
vocab = vocab + ["<mask>", "<sep>", "<a>", "</a>", "<q>", "</q>"]
assert len(vocab) == len(set(vocab))
print("vocab size:", len(vocab))

In [ ]:
test_size = 300       #3000 complete
id_atomic_facts_ds = choose(id_atomic_facts, test_size)   # the atomic facts are the same for all the datasets
ood_atomic_facts_ds = choose(ood_atomic_facts, test_size)  # the atomic facts are the same for all the datasets
# here change

t_3_test_inferred_iid_ds = choose(t_3_test_inferred_iid, test_size) #IID
t_3_test_inferred_facts_ds = choose(t_3_test_inferred_facts, test_size) #OOD

all_atomics = id_atomic_facts + ood_atomic_facts
len(all_atomics)

In [ ]:
# downsampling train_inferred
for phi in [18.0,12.6,9.0,7.2,5.4,3.6]:  #phi is the ratio (Inferred facts / Atomic facts)
    dataset_name = "3_HOP_composition.{}.{}.{}".format(NUM_ENTITY_IN, NUM_RELATION, phi)
    os.makedirs("data_MIO_mask_NEW_correct/{}".format(dataset_name), exist_ok=True)
    t_3_train_inferred_facts_ds = choose(t_3_train_inferred_facts, round(phi * len(id_atomic_facts))) #downsampling the train_inferred_facts (ID)

    
    probes_3 = []
    for item in id_atomic_facts_ds:
        probes_3.append(deepcopy(item))
        probes_3[-1]["type"] = "id_atomic"
    
    for item in ood_atomic_facts_ds:
        probes_3.append(deepcopy(item))
        probes_3[-1]["type"] = "ood_atomic"

    ######### change here for 3 hope 

    for item in choose(t_3_train_inferred_facts_ds, test_size):
        probes_3.append(deepcopy(item))
        probes_3[-1]['type'] = 'train_inferred'

    for item in t_3_test_inferred_iid:
        probes_3.append(deepcopy(item))
        probes_3[-1]['type'] = 'test_inferred_iid'

    for item in t_3_test_inferred_facts_ds:
        probes_3.append(deepcopy(item))
        probes_3[-1]["type"] = "test_inferred_ood"

    # save the dataset Train, Test, Valid
    #train.json = all_atomics + train_inferred_facts_ds
    with open("data_MIO_mask_NEW_correct/{}/train.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(all_atomics + t_3_train_inferred_facts_ds, f)
    #valid.json (only test_inferred_ OOD  why??? for what is that good?) 
    with open("data_MIO_mask_NEW_correct/{}/valid.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(t_3_test_inferred_facts_ds, f)

    # my validation with the test_inferred_iid    
    with open("data_MIO_mask_NEW_correct/{}/valid_iid.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(t_3_test_inferred_iid_ds, f)

    #test.json = probes_3 
    with open("data_MIO_mask_NEW_correct/{}/test.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(probes_3, f)
    # all vocab in the dataset (entities and relations) + special tokens
    with open("data_MIO_mask_NEW_correct/{}/vocab.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(vocab, f)

### 3 HOP split

In [ ]:
test_size= 300  
# same atomic as for split 2
SPLIT_id_atomic_facts_ds = choose(SPLIT_id_atomic_facts, test_size)
SPLIT_ood_atomic_facts_ds = choose(SPLIT_ood_atomic_facts, test_size)
# here change from 2 split
SPLIT_t3_HOP_test_inferred_iid_ds = choose(SPLIT_t3_HOP_test_inferred_iid, test_size) #IID
SPLIT_t3_HOP_test_inferred_facts_ds = choose(SPLIT_t3_HOP_test_inferred_facts, test_size) #OOD

SPLIT_t3_HOP_all_atomics =  SPLIT_id_atomic_facts + SPLIT_ood_atomic_facts
len(SPLIT_t3_HOP_all_atomics)


In [ ]:
# downsampling train_inferred

for phi in [18.0,12.6,9.0,7.2,5.4,3.6]:  #phi is the ratio (Inferred facts / Atomic facts)
    dataset_name = "SPLIT_3_HOP_composition.{}.{}.{}".format(NUM_ENTITY_IN, NUM_RELATION, phi)
    os.makedirs("data_MIO_mask_NEW_correct/{}".format(dataset_name), exist_ok=True)
    SPLIT_t3_HOP_train_inferred_facts_ds = choose(SPLIT_t3_HOP_train_inferred_facts, round(phi * len(SPLIT_id_atomic_facts))) #downsampling the train_inferred_facts (ID)

    SPLIT_t3_HOP_probes = []
    for item in SPLIT_id_atomic_facts_ds:
        SPLIT_t3_HOP_probes.append(deepcopy(item))
        SPLIT_t3_HOP_probes[-1]["type"] = "id_atomic"

    for item in SPLIT_ood_atomic_facts_ds:
        SPLIT_t3_HOP_probes.append(deepcopy(item))
        SPLIT_t3_HOP_probes[-1]["type"] = "ood_atomic"

    ######### change here for 3 hope 

    for item in choose(SPLIT_t3_HOP_train_inferred_facts_ds, test_size):
        SPLIT_t3_HOP_probes.append(deepcopy(item))
        SPLIT_t3_HOP_probes[-1]['type'] = 'train_inferred'

    for item in SPLIT_t3_HOP_test_inferred_iid:
        SPLIT_t3_HOP_probes.append(deepcopy(item))
        SPLIT_t3_HOP_probes[-1]['type'] = 'test_inferred_iid'

    for item in SPLIT_t3_HOP_test_inferred_facts_ds:
        SPLIT_t3_HOP_probes.append(deepcopy(item))
        SPLIT_t3_HOP_probes[-1]["type"] = "test_inferred_ood"

    # save the dataset Train, Test, Valid
    #train.json = all_atomics + train_inferred_facts_ds
    with open("data_MIO_mask_NEW_correct/{}/train.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(SPLIT_t3_HOP_all_atomics + SPLIT_t3_HOP_train_inferred_facts_ds, f)
    #valid.json (only test_inferred_ OOD  why??? for what is that good?) 
    with open("data_MIO_mask_NEW_correct/{}/valid.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(SPLIT_t3_HOP_test_inferred_facts_ds, f)
    
    # my validation with the test_inferred_iid
    with open("data_MIO_mask_NEW_correct/{}/valid_iid.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(SPLIT_t3_HOP_test_inferred_iid_ds, f)
    
    #test.json =
    with open("data_MIO_mask_NEW_correct/{}/test.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(SPLIT_t3_HOP_probes, f)
    # all vocab in the dataset (entities and relations) + special tokens
    with open("data_MIO_mask_NEW_correct/{}/vocab.json".format(dataset_name), "w", encoding='utf-8') as f:
        json.dump(vocab, f)